In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import joblib
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load dataset
df = pd.read_csv("nutrition_dataset_cleaned.csv")

# Fix duplicate goal columns
if 'Goal_Weight Maintenance' in df.columns and 'Goal_Maintenance' in df.columns:
    df['Goal_Maintenance'] = df[['Goal_Maintenance', 'Goal_Weight Maintenance']].max(axis=1)
    df = df.drop(columns=['Goal_Weight Maintenance'])

# Create new feature: BMI
df['BMI'] = df['Weight'] / (df['Height'] / 100) ** 2

# Merge low-sample categories
df['Breakfast Category'] = df['Breakfast Category'].replace(['yogurt-based', 'egg-based'], 'other')
le = LabelEncoder()
df['Breakfast Category Encoded'] = le.fit_transform(df['Breakfast Category'])

# Define features and target
X = df[['Age', 'Gender', 'Height', 'Weight', 'Activity_Lightly Active', 'Activity_Moderately Active',
        'Activity_Sedentary', 'Activity_Very Active', 'Goal_Maintenance', 'Goal_Muscle Gain',
        'Goal_Weight Loss', 'Diet_Omnivore', 'Diet_Vegan', 'Diet_Vegetarian', 'BMI']]
y = df['Breakfast Category Encoded']

# Scale numerical features
scaler = StandardScaler()
X[['Age', 'Height', 'Weight', 'BMI']] = scaler.fit_transform(X[['Age', 'Height', 'Weight', 'BMI']])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7368\2514516355.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['Age', 'Height', 'Weight', 'BMI']] = scaler.fit_transform(X[['Age', 'Height', 'Weight', 'BMI']])


In [5]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Compute class weights
class_counts = Counter(y_train)
total_samples = sum(class_counts.values())
class_weights = {cls: total_samples / (len(class_counts) * count) for cls, count in class_counts.items()}

# Initialize and train Random Forest
rf_model = RandomForestClassifier(random_state=42, n_estimators=100, class_weight=class_weights)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)
print("\nRandom Forest Results:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))

# Feature importance
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': rf_model.feature_importances_}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance)


Random Forest Results:
Accuracy: 0.65
Classification Report:
               precision    recall  f1-score   support

oatmeal-based       0.42      0.33      0.37        24
        other       0.65      0.72      0.68        39
pancake-based       0.00      0.00      0.00         2
   tofu-based       0.81      0.83      0.82        35

     accuracy                           0.65       100
    macro avg       0.47      0.47      0.47       100
 weighted avg       0.64      0.65      0.64       100


Feature Importance:
                       Feature  Importance
0                          Age    0.146769
2                       Height    0.140280
14                         BMI    0.136137
3                       Weight    0.119545
11               Diet_Omnivore    0.109623
9             Goal_Muscle Gain    0.071585
12                  Diet_Vegan    0.067129
10            Goal_Weight Loss    0.051533
7         Activity_Very Active    0.032708
1                       Gender    0.025158
1

In [ ]:
# Save model
joblib.dump(rf_model, "random_forest_breakfast_model.pkl")
print("\nRandom Forest model saved as 'random_forest_breakfast_model.pkl'")